In [1]:
import newspaper
import csv
import pandas as pd

In [2]:
file = 'data/training_dataset.csv'

In [3]:
def urls_from_csv(csv_file, column=None):
    '''
    Takes csv directory and returns list of URLs
    '''
    with open(csv_file, 'r') as f:
        reader = csv.reader(f)
        contents = list(reader)
    
    urls = [line[1] for line in contents[1:]]
    return urls, contents

In [4]:
def urls_to_df(csv_file, column=None):
    '''
    Takes csv directory and returns list of URLs
    '''
    df = pd.read_csv(csv_file)
    df.columns = [x.lower() for x in df.columns]
    urls = list(df['url'])
    return urls, df

In [5]:
urls, contents = urls_from_csv(file)

In [6]:
def remove_newline(text):
    ''' Removes new line and &nbsp characters.
    '''
    text = text.replace('\n', ' ')
    text = text.replace('\xa0', ' ')
    return text

In [7]:
def html_report(link, nlp=False):
    report = {}
    a = newspaper.Article(link)
    a.download()
    a.parse()
    report['domain'] = a.source_url
    report['title'] = a.title
    report['authors'] = a.authors
    report['date_pub'] = a.publish_date
    report['text'] = remove_newline(a.text)
    # tag the type of article
    ## currently default to text but should be able to determine img/video etc
    report['type'] = 'text'
    return report

In [8]:
urls, df = urls_to_df(file)

In [9]:
def scrape_from_urls(urls):
    reports = []
    for url in urls:
        if url[-3:] == 'pdf':
            continue
        else:
            report = html_report(url)
            reports.append(report)
            
    return reports

In [24]:
url = urls[1]

In [25]:
a = newspaper.Article(url)

In [26]:
a.download()

In [27]:
a.parse()

''

In [110]:
keys = report[0].keys()
with open('data.csv', 'w') as f:
    dict_writer = csv.DictWriter(f, fieldnames=keys)
    dict_writer.writeheader()
    dict_writer.writerows(report)